In [1]:
# Deep Convolutional GANs

# Importing the libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

In [2]:
# Setting some hyperparameters
batchSize = 64 # We set the size of the batch.
imageSize = 64 # We set the size of the generated images (64x64).

# Creating the transformations
transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.


In [3]:
# Loading the dataset
dataset = dset.CIFAR10(root = './data', download = True, transform = transform) # We download the training set in the ./data folder and we apply the previous transformations on each image.
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) # We use dataLoader to get the images of the training set batch by batch.

Files already downloaded and verified


In [4]:
# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [5]:
# Defining the generator

class G(nn.Module): # We introduce a class to define the generator.

    def __init__(self): # We introduce the __init__() function that will define the architecture of the generator.
        super(G, self).__init__() # We inherit from the nn.Module tools.
        self.main = nn.Sequential( # We create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False), # We start with an inversed convolution.
            nn.BatchNorm2d(512), # We normalize all the features along the dimension of the batch.
            nn.ReLU(True), # We apply a ReLU rectification to break the linearity.
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(256), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(128), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(64), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.Tanh() # We apply a Tanh rectification to break the linearity and stay between -1 and +1.
        )

    def forward(self, input): # We define the forward function that takes as argument an input that will be fed to the neural network, and that will return the output containing the generated images.
        output = self.main(input) # We forward propagate the signal through the whole neural network of the generator defined by self.main.
        return output # We return the output containing the generated images.

In [6]:
# Creating the generator
netG = G() # We create the generator object.
netG.apply(weights_init) # We initialize all the weights of its neural network.


G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [7]:
# Defining the discriminator

class D(nn.Module): # We introduce a class to define the discriminator.

    def __init__(self): # We introduce the __init__() function that will define the architecture of the discriminator.
        super(D, self).__init__() # We inherit from the nn.Module tools.
        self.main = nn.Sequential( # We create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
            nn.Conv2d(3, 64, 4, 2, 1, bias = False), # We start with a convolution.
            nn.LeakyReLU(0.2, inplace = True), # We apply a LeakyReLU.
            nn.Conv2d(64, 128, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(128), # We normalize all the features along the dimension of the batch.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(128, 256, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(256), # We normalize again.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(256, 512, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(512), # We normalize again.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(512, 1, 4, 1, 0, bias = False), # We add another convolution.
            nn.Sigmoid() # We apply a Sigmoid rectification to break the linearity and stay between 0 and 1.
        )

    def forward(self, input): # We define the forward function that takes as argument an input that will be fed to the neural network, and that will return the output which will be a value between 0 and 1.
        output = self.main(input) # We forward propagate the signal through the whole neural network of the discriminator defined by self.main.
        return output.view(-1) # We return the output which will be a value between 0 and 1.

In [8]:
# Creating the discriminator
netD = D() # We create the discriminator object.
netD.apply(weights_init) # We initialize all the weights of its neural network.


D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [9]:
# Training the DCGANs

criterion = nn.BCELoss() # We create a criterion object that will measure the error between the prediction and the target.
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999)) # We create the optimizer object of the discriminator.
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999)) # We create the optimizer object of the generator.

In [10]:
for epoch in range(10): # We iterate over 10 epochs.

    for i, data in enumerate(dataloader, 0): # We iterate over the images of the dataset.
        
        # 1st Step: Updating the weights of the neural network of the discriminator

        netD.zero_grad() # We initialize to 0 the gradients of the discriminator with respect to the weights.
        
        # Training the discriminator with a real image of the dataset
        real, _ = data # We get a real image of the dataset which will be used to train the discriminator.
        input = Variable(real) # We wrap it in a variable.
        target = Variable(torch.ones(input.size()[0])) # We get the target.
        output = netD(input) # We forward propagate this real image into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errD_real = criterion(output, target) # We compute the loss between the predictions (output) and the target (equal to 1).
        
        # Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1)) # We make a random input vector (noise) of the generator.
        fake = netG(noise) # We forward propagate this random input vector into the neural network of the generator to get some fake generated images.
        target = Variable(torch.zeros(input.size()[0])) # We get the target.
        output = netD(fake.detach()) # We forward propagate the fake generated images into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errD_fake = criterion(output, target) # We compute the loss between the prediction (output) and the target (equal to 0).

        # Backpropagating the total error
        errD = errD_real + errD_fake # We compute the total error of the discriminator.
        errD.backward() # We backpropagate the loss error by computing the gradients of the total error with respect to the weights of the discriminator.
        optimizerD.step() # We apply the optimizer to update the weights according to how much they are responsible for the loss error of the discriminator.

        # 2nd Step: Updating the weights of the neural network of the generator

        netG.zero_grad() # We initialize to 0 the gradients of the generator with respect to the weights.
        target = Variable(torch.ones(input.size()[0])) # We get the target.
        output = netD(fake) # We forward propagate the fake generated images into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errG = criterion(output, target) # We compute the loss between the prediction (output between 0 and 1) and the target (equal to 1).
        errG.backward() # We backpropagate the loss error by computing the gradients of the total error with respect to the weights of the generator.
        optimizerG.step() # We apply the optimizer to update the weights according to how much they are responsible for the loss error of the generator.
        
        # 3rd Step: Printing the losses and saving the real images and the generated images of the minibatch every 100 steps

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 10, i, len(dataloader), errD.data[0], errG.data[0])) # We print les losses of the discriminator (Loss_D) and the generator (Loss_G).
        if i % 100 == 0: # Every 100 steps:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True) # We save the real images of the minibatch.
            fake = netG(noise) # We get our fake generated images.
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True) # We also save the fake generated images of the minibatch.

/home/aman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[0/10][0/782] Loss_D: 1.5422 Loss_G: 5.9273
[0/10][1/782] Loss_D: 1.0842 Loss_G: 5.5251
[0/10][2/782] Loss_D: 0.9837 Loss_G: 5.6630
[0/10][3/782] Loss_D: 0.9427 Loss_G: 6.5886
[0/10][4/782] Loss_D: 0.9654 Loss_G: 5.9732
[0/10][5/782] Loss_D: 1.1369 Loss_G: 6.8875
[0/10][6/782] Loss_D: 0.7487 Loss_G: 7.4728
[0/10][7/782] Loss_D: 0.7711 Loss_G: 7.4983
[0/10][8/782] Loss_D: 0.8464 Loss_G: 7.1007
[0/10][9/782] Loss_D: 0.6074 Loss_G: 8.6638
[0/10][10/782] Loss_D: 0.5220 Loss_G: 7.3551
[0/10][11/782] Loss_D: 0.6265 Loss_G: 11.2067
[0/10][12/782] Loss_D: 0.4373 Loss_G: 8.2526
[0/10][13/782] Loss_D: 0.7168 Loss_G: 10.2384
[0/10][14/782] Loss_D: 0.4083 Loss_G: 9.3862
[0/10][15/782] Loss_D: 0.7649 Loss_G: 13.0581
[0/10][16/782] Loss_D: 0.3868 Loss_G: 9.3225
[0/10][17/782] Loss_D: 1.1702 Loss_G: 14.9157
[0/10][18/782] Loss_D: 0.6234 Loss_G: 11.6247
[0/10][19/782] Loss_D: 0.3436 Loss_G: 7.2775
[0/10][20/782] Loss_D: 1.9940 Loss_G: 17.9189
[0/10][21/782] Loss_D: 0.3882 Loss_G: 18.2793
[0/10][22/782

[0/10][179/782] Loss_D: 0.4219 Loss_G: 3.1653
[0/10][180/782] Loss_D: 0.5123 Loss_G: 4.9594
[0/10][181/782] Loss_D: 0.4750 Loss_G: 2.9785
[0/10][182/782] Loss_D: 0.4557 Loss_G: 4.2139
[0/10][183/782] Loss_D: 0.3374 Loss_G: 4.9329
[0/10][184/782] Loss_D: 0.4002 Loss_G: 3.4939
[0/10][185/782] Loss_D: 0.4416 Loss_G: 5.5649
[0/10][186/782] Loss_D: 0.5648 Loss_G: 3.1419
[0/10][187/782] Loss_D: 0.4270 Loss_G: 6.0960
[0/10][188/782] Loss_D: 0.3656 Loss_G: 4.1020
[0/10][189/782] Loss_D: 0.4663 Loss_G: 5.8539
[0/10][190/782] Loss_D: 0.3403 Loss_G: 4.3333
[0/10][191/782] Loss_D: 0.4085 Loss_G: 4.5730
[0/10][192/782] Loss_D: 0.3316 Loss_G: 5.5824
[0/10][193/782] Loss_D: 0.4354 Loss_G: 3.1067
[0/10][194/782] Loss_D: 0.5948 Loss_G: 8.8927
[0/10][195/782] Loss_D: 0.8485 Loss_G: 4.2918
[0/10][196/782] Loss_D: 0.1542 Loss_G: 3.5835
[0/10][197/782] Loss_D: 0.8097 Loss_G: 9.1443
[0/10][198/782] Loss_D: 0.9832 Loss_G: 4.5823
[0/10][199/782] Loss_D: 0.3092 Loss_G: 5.3486
[0/10][200/782] Loss_D: 0.2513 Los

[0/10][357/782] Loss_D: 0.4661 Loss_G: 3.9001
[0/10][358/782] Loss_D: 0.3432 Loss_G: 3.7099
[0/10][359/782] Loss_D: 0.5047 Loss_G: 4.9571
[0/10][360/782] Loss_D: 0.3351 Loss_G: 4.4330
[0/10][361/782] Loss_D: 0.3315 Loss_G: 3.1814
[0/10][362/782] Loss_D: 0.3534 Loss_G: 4.6280
[0/10][363/782] Loss_D: 0.2631 Loss_G: 4.5820
[0/10][364/782] Loss_D: 0.2289 Loss_G: 3.7915
[0/10][365/782] Loss_D: 0.2072 Loss_G: 4.1770
[0/10][366/782] Loss_D: 0.1509 Loss_G: 4.1547
[0/10][367/782] Loss_D: 0.3218 Loss_G: 3.5298
[0/10][368/782] Loss_D: 0.2436 Loss_G: 4.3421
[0/10][369/782] Loss_D: 0.3914 Loss_G: 4.3290
[0/10][370/782] Loss_D: 0.1772 Loss_G: 4.3825
[0/10][371/782] Loss_D: 0.3756 Loss_G: 4.1007
[0/10][372/782] Loss_D: 0.7017 Loss_G: 2.8943
[0/10][373/782] Loss_D: 0.5832 Loss_G: 5.8351
[0/10][374/782] Loss_D: 0.5529 Loss_G: 4.0538
[0/10][375/782] Loss_D: 0.5048 Loss_G: 4.3357
[0/10][376/782] Loss_D: 0.6938 Loss_G: 4.4895
[0/10][377/782] Loss_D: 0.4470 Loss_G: 2.8992
[0/10][378/782] Loss_D: 0.9752 Los

[0/10][536/782] Loss_D: 0.9464 Loss_G: 9.3068
[0/10][537/782] Loss_D: 0.2385 Loss_G: 4.9707
[0/10][538/782] Loss_D: 0.2794 Loss_G: 3.7901
[0/10][539/782] Loss_D: 0.5812 Loss_G: 6.8139
[0/10][540/782] Loss_D: 0.3730 Loss_G: 5.7624
[0/10][541/782] Loss_D: 0.4132 Loss_G: 4.0234
[0/10][542/782] Loss_D: 0.5564 Loss_G: 7.5566
[0/10][543/782] Loss_D: 0.5284 Loss_G: 4.4464
[0/10][544/782] Loss_D: 0.2624 Loss_G: 3.2655
[0/10][545/782] Loss_D: 0.4709 Loss_G: 7.6924
[0/10][546/782] Loss_D: 0.1961 Loss_G: 7.8886
[0/10][547/782] Loss_D: 0.4872 Loss_G: 3.6319
[0/10][548/782] Loss_D: 0.2766 Loss_G: 3.2945
[0/10][549/782] Loss_D: 0.3364 Loss_G: 6.2252
[0/10][550/782] Loss_D: 0.1111 Loss_G: 6.6359
[0/10][551/782] Loss_D: 0.1462 Loss_G: 4.3130
[0/10][552/782] Loss_D: 0.1655 Loss_G: 4.0450
[0/10][553/782] Loss_D: 0.1915 Loss_G: 4.6636
[0/10][554/782] Loss_D: 0.1651 Loss_G: 4.9734
[0/10][555/782] Loss_D: 0.3855 Loss_G: 3.3639
[0/10][556/782] Loss_D: 0.4325 Loss_G: 6.0166
[0/10][557/782] Loss_D: 0.3106 Los

...

[8/10][738/782] Loss_D: 0.4117 Loss_G: 3.2666
[8/10][739/782] Loss_D: 0.3298 Loss_G: 2.4567
[8/10][740/782] Loss_D: 0.5116 Loss_G: 4.2900
[8/10][741/782] Loss_D: 0.2418 Loss_G: 3.5242
[8/10][742/782] Loss_D: 0.2441 Loss_G: 3.0122
[8/10][743/782] Loss_D: 0.4338 Loss_G: 3.8871
[8/10][744/782] Loss_D: 0.2204 Loss_G: 4.6697
[8/10][745/782] Loss_D: 0.2795 Loss_G: 2.6366
[8/10][746/782] Loss_D: 0.5380 Loss_G: 7.5953
[8/10][747/782] Loss_D: 1.9879 Loss_G: 1.2650
[8/10][748/782] Loss_D: 1.3675 Loss_G: 6.3078
[8/10][749/782] Loss_D: 2.8068 Loss_G: 0.5520
[8/10][750/782] Loss_D: 1.6496 Loss_G: 3.6704
[8/10][751/782] Loss_D: 1.8511 Loss_G: 0.5120
[8/10][752/782] Loss_D: 1.5003 Loss_G: 3.0700
[8/10][753/782] Loss_D: 1.1727 Loss_G: 1.5180
[8/10][754/782] Loss_D: 1.2533 Loss_G: 2.0630
[8/10][755/782] Loss_D: 0.9479 Loss_G: 1.9151
[8/10][756/782] Loss_D: 0.7732 Loss_G: 2.1017
[8/10][757/782] Loss_D: 0.7864 Loss_G: 2.1137
[8/10][758/782] Loss_D: 0.9431 Loss_G: 2.2702
[8/10][759/782] Loss_D: 1.2298 Los

[9/10][137/782] Loss_D: 0.7476 Loss_G: 2.1432
[9/10][138/782] Loss_D: 0.8216 Loss_G: 1.8549
[9/10][139/782] Loss_D: 0.8493 Loss_G: 2.5337
[9/10][140/782] Loss_D: 0.8001 Loss_G: 1.3619
[9/10][141/782] Loss_D: 0.8610 Loss_G: 2.7668
[9/10][142/782] Loss_D: 0.6785 Loss_G: 2.4742
[9/10][143/782] Loss_D: 0.6455 Loss_G: 1.6326
[9/10][144/782] Loss_D: 0.6526 Loss_G: 2.7221
[9/10][145/782] Loss_D: 0.5873 Loss_G: 2.4808
[9/10][146/782] Loss_D: 0.7317 Loss_G: 1.6447
[9/10][147/782] Loss_D: 0.7576 Loss_G: 3.0791
[9/10][148/782] Loss_D: 0.7268 Loss_G: 1.9668
[9/10][149/782] Loss_D: 0.3788 Loss_G: 2.4445
[9/10][150/782] Loss_D: 0.7240 Loss_G: 1.7608
[9/10][151/782] Loss_D: 0.7071 Loss_G: 2.5091
[9/10][152/782] Loss_D: 0.8570 Loss_G: 1.5464
[9/10][153/782] Loss_D: 0.8739 Loss_G: 2.4362
[9/10][154/782] Loss_D: 0.8091 Loss_G: 1.8805
[9/10][155/782] Loss_D: 0.5810 Loss_G: 3.2864
[9/10][156/782] Loss_D: 0.5991 Loss_G: 2.1974
[9/10][157/782] Loss_D: 0.8303 Loss_G: 1.4571
[9/10][158/782] Loss_D: 0.6646 Los

[9/10][316/782] Loss_D: 0.1851 Loss_G: 7.1358
[9/10][317/782] Loss_D: 0.2581 Loss_G: 5.0670
[9/10][318/782] Loss_D: 0.0619 Loss_G: 3.4723
[9/10][319/782] Loss_D: 0.0333 Loss_G: 4.1440
[9/10][320/782] Loss_D: 0.0507 Loss_G: 4.7992
[9/10][321/782] Loss_D: 0.1662 Loss_G: 8.2562
[9/10][322/782] Loss_D: 0.9710 Loss_G: 4.5493
[9/10][323/782] Loss_D: 1.7792 Loss_G: 0.0055
[9/10][324/782] Loss_D: 6.7652 Loss_G: 5.1826
[9/10][325/782] Loss_D: 1.6997 Loss_G: 2.0002
[9/10][326/782] Loss_D: 0.6255 Loss_G: 1.1618
[9/10][327/782] Loss_D: 0.9006 Loss_G: 3.6818
[9/10][328/782] Loss_D: 0.6625 Loss_G: 2.1726
[9/10][329/782] Loss_D: 0.5021 Loss_G: 1.6771
[9/10][330/782] Loss_D: 0.6387 Loss_G: 2.6510
[9/10][331/782] Loss_D: 0.7234 Loss_G: 2.1275
[9/10][332/782] Loss_D: 0.5606 Loss_G: 2.5898
[9/10][333/782] Loss_D: 0.6557 Loss_G: 1.8591
[9/10][334/782] Loss_D: 0.6120 Loss_G: 3.1783
[9/10][335/782] Loss_D: 0.4461 Loss_G: 2.6710
[9/10][336/782] Loss_D: 0.5048 Loss_G: 1.8042
[9/10][337/782] Loss_D: 0.5993 Los

[9/10][495/782] Loss_D: 0.5197 Loss_G: 2.9074
[9/10][496/782] Loss_D: 0.6360 Loss_G: 2.1348
[9/10][497/782] Loss_D: 0.5029 Loss_G: 3.3581
[9/10][498/782] Loss_D: 0.8807 Loss_G: 1.7376
[9/10][499/782] Loss_D: 0.8054 Loss_G: 4.5729
[9/10][500/782] Loss_D: 0.5789 Loss_G: 3.3277
[9/10][501/782] Loss_D: 0.3243 Loss_G: 3.2055
[9/10][502/782] Loss_D: 0.3842 Loss_G: 3.2914
[9/10][503/782] Loss_D: 0.3019 Loss_G: 3.4410
[9/10][504/782] Loss_D: 0.3348 Loss_G: 3.6282
[9/10][505/782] Loss_D: 0.2035 Loss_G: 4.1641
[9/10][506/782] Loss_D: 0.3278 Loss_G: 3.0601
[9/10][507/782] Loss_D: 0.4058 Loss_G: 2.1503
[9/10][508/782] Loss_D: 0.8592 Loss_G: 5.9830
[9/10][509/782] Loss_D: 0.6889 Loss_G: 3.1064
[9/10][510/782] Loss_D: 0.3391 Loss_G: 4.0786
[9/10][511/782] Loss_D: 0.2232 Loss_G: 3.9324
[9/10][512/782] Loss_D: 0.2519 Loss_G: 4.6732
[9/10][513/782] Loss_D: 0.1984 Loss_G: 3.8133
[9/10][514/782] Loss_D: 0.1083 Loss_G: 4.0762
[9/10][515/782] Loss_D: 0.1739 Loss_G: 3.9407
[9/10][516/782] Loss_D: 0.1229 Los

[9/10][674/782] Loss_D: 0.3982 Loss_G: 2.1554
[9/10][675/782] Loss_D: 0.6006 Loss_G: 5.1724
[9/10][676/782] Loss_D: 0.9724 Loss_G: 1.2856
[9/10][677/782] Loss_D: 1.6684 Loss_G: 7.0414
[9/10][678/782] Loss_D: 3.3827 Loss_G: 0.9120
[9/10][679/782] Loss_D: 1.3129 Loss_G: 4.2930
[9/10][680/782] Loss_D: 0.5983 Loss_G: 2.7036
[9/10][681/782] Loss_D: 0.5504 Loss_G: 2.3500
[9/10][682/782] Loss_D: 0.3262 Loss_G: 3.3311
[9/10][683/782] Loss_D: 0.5078 Loss_G: 2.2211
[9/10][684/782] Loss_D: 0.2842 Loss_G: 3.6866
[9/10][685/782] Loss_D: 0.3287 Loss_G: 3.7416
[9/10][686/782] Loss_D: 0.6339 Loss_G: 0.9284
[9/10][687/782] Loss_D: 1.0470 Loss_G: 7.0395
[9/10][688/782] Loss_D: 0.9799 Loss_G: 2.6269
[9/10][689/782] Loss_D: 0.4403 Loss_G: 3.7708
[9/10][690/782] Loss_D: 0.3715 Loss_G: 4.1016
[9/10][691/782] Loss_D: 0.2420 Loss_G: 2.6907
[9/10][692/782] Loss_D: 0.4620 Loss_G: 5.3860
[9/10][693/782] Loss_D: 0.2723 Loss_G: 3.9882
[9/10][694/782] Loss_D: 0.1263 Loss_G: 3.4897
[9/10][695/782] Loss_D: 0.3786 Los